In [ ]:
#
# Licensed to the Apache Software Foundation (ASF) under one or more
# contributor license agreements.  See the NOTICE file distributed with
# this work for additional information regarding copyright ownership.
# The ASF licenses this file to You under the Apache License, Version 2.0
# (the "License"); you may not use this file except in compliance with
# the License.  You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#

"""A minimalist word-counting workflow that counts words in Shakespeare.

This is the first in a series of successively more detailed 'word count'
examples.

Next, see the wordcount pipeline, then the wordcount_debugging pipeline, for
more detailed examples that introduce additional concepts.

Concepts:

1. Reading data from text files
2. Specifying 'inline' transforms
3. Counting a PCollection
4. Writing data to Cloud Storage as text files

To execute this pipeline locally, first edit the code to specify the output
location. Output location could be a local file path or an output prefix
on GCS. (Only update the output location marked with the first CHANGE comment.)

To execute this pipeline remotely, first edit the code to set your project ID,
runner type, the staging location, the temp location, and the output location.
The specified GCS bucket(s) must already exist. (Update all the places marked
with a CHANGE comment.)

Then, run the pipeline as described in the README. It will be deployed and run
using the Google Cloud Dataflow Service. No args are required to run the
pipeline. You can see the results in your output bucket in the GCS browser.
"""

# pytype: skip-file

# beam-playground:
#   name: WordCountMinimal
#   description: An example that counts words in Shakespeare's works.
#   multifile: false
#   pipeline_options: --output output.txt
#   context_line: 74
#   categories:
#     - IO
#     - Core Transforms
#     - Flatten
#     - Options
#     - Combiners
#     - Quickstart
#   complexity: MEDIUM
#   tags:
#     - count
#     - strings
#     - hellobeam

import argparse
import logging
import re

import apache_beam as beam
from apache_beam.io import ReadFromText
from apache_beam.io import WriteToText
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.options.pipeline_options import SetupOptions


def main(argv=None, save_main_session=True):
  """Main entry point; defines and runs the wordcount pipeline."""

  parser = argparse.ArgumentParser()
  parser.add_argument(
      '--input',
      dest='input',
      default='gs://dataflow-samples/shakespeare/kinglear.txt',
      help='Input file to process.')
  parser.add_argument(
      '--output',
      dest='output',
      # CHANGE 1/6: (OPTIONAL) The Google Cloud Storage path is required
      # for outputting the results.
      default='gs://YOUR_OUTPUT_BUCKET/AND_OUTPUT_PREFIX',
      help='Output file to write results to.')

  # If you use DataflowRunner, below options can be passed:
  #   CHANGE 2/6: (OPTIONAL) Change this to DataflowRunner to
  #   run your pipeline on the Google Cloud Dataflow Service.
  #   '--runner=DirectRunner',
  #   CHANGE 3/6: (OPTIONAL) Your project ID is required in order to
  #   run your pipeline on the Google Cloud Dataflow Service.
  #   '--project=SET_YOUR_PROJECT_ID_HERE',
  #   CHANGE 4/6: (OPTIONAL) The Google Cloud region (e.g. us-central1)
  #   is required in order to run your pipeline on the Google Cloud
  #   Dataflow Service.
  #   '--region=SET_REGION_HERE',
  #   CHANGE 5/6: Your Google Cloud Storage path is required for staging local
  #   files.
  #   '--staging_location=gs://YOUR_BUCKET_NAME/AND_STAGING_DIRECTORY',
  #   CHANGE 6/6: Your Google Cloud Storage path is required for temporary
  #   files.
  #   '--temp_location=gs://YOUR_BUCKET_NAME/AND_TEMP_DIRECTORY',
  #   '--job_name=your-wordcount-job',
  known_args, pipeline_args = parser.parse_known_args(argv) #return namespace of know argument and a list of additional argument

  # We use the save_main_session option because one or more DoFn's in this
  # workflow rely on global context (e.g., a module imported at module level).
  pipeline_options = PipelineOptions(pipeline_args)
  #change sasve_main_session to True because workder can import the same module from the main session
  pipeline_options.view_as(SetupOptions).save_main_session = save_main_session
  with beam.Pipeline(options=pipeline_options) as p:

    # Read the text file[pattern] into a PCollection.
    lines = p | ReadFromText(known_args.input)

    # Count the occurrences of each word.
    counts = (
        lines
        | 'Split' >> (
            beam.FlatMap(
                lambda x: re.findall(r'[A-Za-z\']+', x)).with_output_types(str))
        | 'PairWithOne' >> beam.Map(lambda x: (x, 1))
        | 'GroupAndSum' >> beam.CombinePerKey(sum))

    # Format the counts into a PCollection of strings.
    def format_result(word_count):
      (word, count) = word_count
      return '%s: %s' % (word, count)

    output = counts | 'Format' >> beam.Map(format_result)

    # Write the output using a "Write" transform that has side effects.
    # pylint: disable=expression-not-assigned
    output | WriteToText(known_args.output)


if __name__ == '__main__':
  logging.getLogger().setLevel(logging.INFO)
  main()

In [ ]:
#let check the packages we need to use
#we use package like: re, logging, argparse, apache_beam, 
import re
import logging
import argparse
import apache_beam as beam
from apache_beam.io import ReadFromText
from apache_beam.io import WriteToText
from apache_beam.options.pipeline_options import PipelineOptions, SetupOptions

In [ ]:
#script to run this python scripts
!python -m apache_beam.examples.wordcount_minimal --input YOUR_INPUT_FILE --output counts

**Summary note:**
- package use: re, argparse, logging, apache_beam.
- function here is define inside pipelie context
- using object like: beam.FlatMap(), beam.Map(), beam.CombinePerKey()
- logic: 
    - read from text file line by line. ReadFromText
    - for reach line parse word and return a list of words
    - for each word in the list return list of tuple (word, 1)
    - take a list of tuple (word, 1) to compute sum per key with beam.CombinePerKey
    - unpack the result and then write to file system.

**Note about apache beam api:**
- ReadFromText(file_pattern): parse text file and return each line
- beam.FlatMap(fn): take an element in put and return iteratable
- beam.Map(fn): take a single element and return a single element
- beam.CombinePerKey(fn): apply CombineFns to Pcollecton
- note: where we use ReadFromText -> FlatMap -> Map -> CombinePerKey -> Map -> WriteToText